# Определение текущего уровня потребительской лояльности среди клиентов из России

## Описание проекта

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ.<br>  
Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.<br>   
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».<br>  
Компания провела опрос и попросила вас подготовить дашборд с его итогами. <br>  
Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.<br>   
Чтобы оценить результаты опроса, оценки обычно делят на три группы:<br>
9-10 баллов — «cторонники» (англ. promoters);<br>
7-8 баллов — «нейтралы» (англ. passives);<br>
0-6 баллов — «критики» (англ. detractors).<br>  
Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

## Цель исследования

Получение ответов на вопросы:<br>  
1. Как распределены участники опроса по возрасту, полу и возрасту? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
2. Какие группы пользователей наиболее лояльны к сервису? Какие менее?
3. Какой общий NPS среди всех опрошенных?
4. Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

## Этапы исследования

1. Выгрузка данных
2. Создание дашборда в Tableau
3. Подготовка презентации с ответами на вопросы

### Выгрузка данных

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
query = """
SELECT u.user_id,
       u.lt_day,
       CASE 
           WHEN u.lt_day <= 365 THEN 'Новый'
           ELSE 'Старый'
           END AS is_new,
       u.age,
       CASE 
           WHEN u.gender_segment == 1 THEN 'Женщина'
           WHEN u.gender_segment == 0 THEN 'Мужчина'
           ELSE 'x'
           END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       location.country,
       location.city,
       SUBSTR(age_segment.title, 4) AS age_segment,
       SUBSTR(traffic_segment.title, 4) AS traffic_segment,
       SUBSTR(lifetime_segment.title, 4) AS lifetime_segment,
       u.nps_score,
       CASE 
           WHEN u.nps_score >= 9 AND u.nps_score <= 10 THEN 'Cторонник'
           WHEN u.nps_score >= 7 AND u.nps_score <= 8 THEN 'Нейтрал'
           ELSE 'Критик'
           END AS nps_group
FROM user AS u
JOIN location ON u.location_id = location.location_id
JOIN age_segment ON u.age_gr_id = age_segment.age_gr_id
JOIN traffic_segment ON u.tr_gr_id = traffic_segment.tr_gr_id
JOIN lifetime_segment ON u.lt_gr_id = lifetime_segment.lt_gr_id;
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Старый,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,Cторонник
1,A001WF,2344,Старый,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,Cторонник
2,A003Q7,467,Старый,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,Cторонник


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

**Описание таблицы** <br>

* user_id	- Идентификатор клиента
* lt_day	- Количество дней «жизни» клиента
* is_new	- Поле хранит информацию о том, является ли клиент новым
* age	- Возраст
* gender_segment	- Пол
* os_name	- Тип операционной системы
* cpe_type_name	- Тип устройства
* country	- Страна проживания
* city	- Город проживания
* age_segment	- Возрастной сегмент
* traffic_segment	- Сегмент по объёму потребляемого трафика
* lifetime_segment	- Сегмент по количеству дней «жизни»
* nps_score	- Оценка клиента в NPS-опросе
* nps_group	- Поле хранит информацию о том, к какой группе относится оценка клиента в опросе

### Создание дашборда в Tableau

Дашборд:<br>
    
https://public.tableau.com/app/profile/valeriya.stroganova/viz/NPS_16835661621280/NPS_2?publish=yes

### Презентация с ответами на вопросы

<div class="alert alert-info" style="border:solid blue 2px; padding: 20px"> 
Презентация:<br>
    
https://disk.yandex.ru/i/6IyvEbfJrN_vFw